### Utility Functions

In [ ]:
import os
import glob
import itertools
import random
import socket
import struct
import boto3


CLIENT = boto3.client('s3')

def upload(table, bucket, path):
    assert os.path.isdir(table)
    for item in itertools.chain(glob.glob(table + '/**'), glob.glob(table + '/.**')):
        if not os.path.isfile(item):
            upload(item, bucket, path + '/' + os.path.basename(item))
        else:
            CLIENT.upload_file(
                item,
                bucket,
                os.path.join(path, os.path.basename(item))
            )


def ipv4():
    return socket.inet_ntoa(struct.pack('>I', random.randint(1, 0xffffffff)))


def port():
    return random.randrange(0, 65536)

### Prepare CSV File

In [ ]:
import csv


with open('../../data/security.csv', 'w', newline='') as csvfile:
    writer = csv.writer(
        csvfile,
        delimiter=',',
    )
    writer.writerow(['src_ip', 'src_port', 'dst_ip', 'dst_port'])
    for _ in range(10_000):
        writer.writerow([ipv4(), port(), ipv4(), port()])

### Setup Spark Session

In [6]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip


builder = SparkSession.builder.appName('CreateDeltaTables') \
    .config(
        'spark.jars.packages',
        'io.delta:delta-core_2.12:2.2.0') \
    .config(
        'spark.sql.extensions',
        'io.delta.sql.DeltaSparkSessionExtension') \
    .config(
        'spark.sql.catalog.spark_catalog',
        'org.apache.spark.sql.delta.catalog.DeltaCatalog')

spark = configure_spark_with_delta_pip(builder).getOrCreate()

### Load CSV

In [ ]:
security = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load('../../data/security.csv')
security.show(n=5, truncate=False, vertical=True)

### Split Delta Table

In [ ]:
n_splits = 10
each_len = security.count() // n_splits
copied = security

i = 0
while i < n_splits:
    chunk = copied.limit(each_len)
    copied = copied.subtract(chunk)
    if i == 0:
        chunk.write \
            .format('delta') \
            .save('../../data/security-table')
    else:
        chunk.write \
            .mode('append') \
            .format('delta') \
            .save('../../data/security-table')
    i += 1

### Partition by Range

In [11]:
security = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load('../../data/security.csv')
sorted_security = security \
    .repartition('src_ip').sortWithinPartitions("date", "id")
security.write \
  .format('delta') \
  .mode('overwrite') \
  .option('overwriteSchema', 'true') \
  .option('path', '../../data/security-table-part-by-range') \
  .partitionBy(['src_ip', 'src_port', 'dst_ip']) \
  .saveAsTable('`security-table-part-by-range`')

ERROR:root:KeyboardInterrupt while sending command.>                (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/shin/.local/share/virtualenvs/sharing-examples-hEeTnWv9/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shin/.local/share/virtualenvs/sharing-examples-hEeTnWv9/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 